## ここで開発していこう

## 01 基本のホッケーゲーム
#### GPTに書かせた

⭐️ゲーム概要

・←,→キー(一人目)、a,dキー(二人目)で操作

・Qキーで終了

In [ ]:
import pygame
import sys

# ゲームの設定
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
PADDLE_WIDTH, PADDLE_HEIGHT = 20, 100
PUCK_RADIUS = 15

# 初期化
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('エアホッケー')
clock = pygame.time.Clock()

# パドルの設定
#paddle1 = pygame.Rect(WIDTH // 2 - PADDLE_WIDTH // 2, HEIGHT - 30, PADDLE_WIDTH, PADDLE_HEIGHT)
paddle1 = pygame.Rect(PADDLE_WIDTH, PADDLE_HEIGHT, WIDTH // 2 - PADDLE_WIDTH // 2, HEIGHT - 30)
#paddle2 = pygame.Rect(WIDTH // 2 - PADDLE_WIDTH // 2, 10, PADDLE_WIDTH, PADDLE_HEIGHT)
paddle2 = pygame.Rect( PADDLE_WIDTH, PADDLE_HEIGHT, WIDTH // 2 - PADDLE_WIDTH // 2, 10)

# パックの設定
puck = pygame.Rect(WIDTH // 2 - PUCK_RADIUS, HEIGHT // 2 - PUCK_RADIUS, PUCK_RADIUS * 2, PUCK_RADIUS * 2)
puck_speed = [5, 5]

# ゲームループ
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                pygame.quit()
                sys.exit()

    # キー操作
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        paddle1.y -= 10
    if keys[pygame.K_RIGHT]:
        paddle1.y += 10
    if keys[pygame.K_a]:
        paddle2.y -= 10
    if keys[pygame.K_d]:
        paddle2.y += 10

    # パドルの位置を画面内に制限
    paddle1.x = max(0, min(WIDTH - PADDLE_WIDTH, paddle1.x))
    paddle2.x = max(0, min(WIDTH - PADDLE_WIDTH, paddle2.x))

    # パックの移動
    puck.x += puck_speed[0]
    puck.y += puck_speed[1]

    # パックの衝突処理
    if puck.left <= 0 or puck.right >= WIDTH:
        puck_speed[0] = -puck_speed[0]
    if puck.top <= 0 or puck.bottom >= HEIGHT:
        puck_speed[1] = -puck_speed[1]

    # パックとパドルの衝突処理
    if puck.colliderect(paddle1) or puck.colliderect(paddle2):
        puck_speed[1] = -puck_speed[1]

    # 画面の描画
    screen.fill(BLACK)
    pygame.draw.rect(screen, WHITE, paddle1)
    pygame.draw.rect(screen, WHITE, paddle2)
    pygame.draw.ellipse(screen, RED, puck)
    pygame.draw.aaline(screen, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))

    pygame.display.flip()
    clock.tick(60)


# 02 横向きの対戦型ゲームにする
#### ほぼ一緒

⭐️ゲーム概要

・↑,↓キー(一人目)、S,Xキー(二人目)で操作

・Qキーで終了

In [ ]:
import pygame
import sys

# ゲームの初期化
pygame.init()


# 定数の設定
WIDTH, HEIGHT = 800, 400
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PADDLE_WIDTH, PADDLE_HEIGHT = 10, 100
PUCK_RADIUS = 10
FPS = 60

# 画面の作成
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('エアホッケー')

# パドルとパックのクラス
class Paddle(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((PADDLE_WIDTH, PADDLE_HEIGHT))
        self.image.fill(BLACK)
        self.rect = self.image.get_rect(topleft=(x, y))

    def move(self, y):
        self.rect.y += y
        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT

class Puck(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((PUCK_RADIUS * 2, PUCK_RADIUS * 2), pygame.SRCALPHA)
        pygame.draw.circle(self.image, BLACK, (PUCK_RADIUS, PUCK_RADIUS), PUCK_RADIUS)
        self.rect = self.image.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        self.speed_x = 4
        self.speed_y = 4

    def update(self):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        if self.rect.top <= 0 or self.rect.bottom >= HEIGHT:
            self.speed_y = -self.speed_y

        if self.rect.left <= 0 or self.rect.right >= WIDTH:
            self.speed_x = -self.speed_x

# インスタンスの作成
paddle1 = Paddle(10, HEIGHT // 2 - PADDLE_HEIGHT // 2)
paddle2 = Paddle(WIDTH - 20, HEIGHT // 2 - PADDLE_HEIGHT // 2)
puck = Puck()

all_sprites = pygame.sprite.Group()
all_sprites.add(paddle1, paddle2, puck)

# メインゲームループ
clock = pygame.time.Clock()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        paddle1.move(-5)
    if keys[pygame.K_s]:
        paddle1.move(5)
    if keys[pygame.K_UP]:
        paddle2.move(-5)
    if keys[pygame.K_DOWN]:
        paddle2.move(5)

    puck.update()

    # 衝突処理
    if pygame.sprite.collide_rect(puck, paddle1) or pygame.sprite.collide_rect(puck, paddle2):
        puck.speed_x = -puck.speed_x

    screen.fill(WHITE)
    all_sprites.draw(screen)
    pygame.display.flip()
    clock.tick(FPS)


# 03 手の座標取得

#### 座標を取得するだけ

カメラの画像を表示させずに手の位置座標のみ取得する

・正規化してない値(cx, cy)

・正規化した値(hx, hy)

In [ ]:
import cv2
import mediapipe as mp
import pygame
import numpy as np

# 定数の設定
WIDTH, HEIGHT = 800, 400
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PADDLE_WIDTH, PADDLE_HEIGHT = 10, 100
PUCK_RADIUS = 10
FPS = 60

# MediaPipeのセットアップ
mp_hands = mp.solutions.hands

# Handsモジュールの初期化
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# ビデオキャプチャの初期化
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("無効なフレームを取得しました。")
        break

    # 画像の反転とRGB変換
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 手のランドマークの検出
    results = hands.process(image_rgb)

    # 手のランドマークの座標を取得
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, landmark in enumerate(hand_landmarks.landmark):
                height, width, _ = image.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)
                hx = np.interp(cx, [0, 3000], [width, 0]) #x座標を正規化
                hy = np.interp(cy, [0, 3000], [0, height]) #y座標を正規化
               
                print(f'ランドマークID {id}: cx={cx}, cy={cy}, hx={hx}, hy={hy}')
    
    # 'q'キーが押された場合はループを終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# リソースの解放
cap.release()
cv2.destroyAllWindows()


# 04　手の座標をパドルにくっつける

#### 1人だけ

手の座標のy軸の値だけパドルのy座標にくっつけている。

In [ ]:
import pygame
import sys
import cv2
import mediapipe as mp
import numpy as np

# ゲームの初期化
pygame.init()

# 定数の設定
WIDTH, HEIGHT = 800, 400
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PADDLE_WIDTH, PADDLE_HEIGHT = 10, 100
PUCK_RADIUS = 10
FPS = 120

# 画面の作成
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('エアホッケー')

# パドルとパックのクラス
class Paddle(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((PADDLE_WIDTH, PADDLE_HEIGHT))
        self.image.fill(BLACK)
        self.rect = self.image.get_rect(topleft=(x, y))

    def move(self, y):
        self.rect.y = y
        if self.rect.top < 0:
            self.rect.top = 0
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT

class Puck(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((PUCK_RADIUS * 2, PUCK_RADIUS * 2), pygame.SRCALPHA)
        pygame.draw.circle(self.image, BLACK, (PUCK_RADIUS, PUCK_RADIUS), PUCK_RADIUS)
        self.rect = self.image.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        self.speed_x = 4
        self.speed_y = 4

    def update(self):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y

        if self.rect.top <= 0 or self.rect.bottom >= HEIGHT:
            self.speed_y = -self.speed_y

        if self.rect.left <= 0 or self.rect.right >= WIDTH:
            self.speed_x = -self.speed_x

# MediaPipeのセットアップ
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# ビデオキャプチャの初期化
cap = cv2.VideoCapture(0)

# インスタンスの作成
paddle1 = Paddle(10, HEIGHT // 2 - PADDLE_HEIGHT // 2)
paddle2 = Paddle(WIDTH - 20, HEIGHT // 2 - PADDLE_HEIGHT // 2)
puck = Puck()

all_sprites = pygame.sprite.Group()
all_sprites.add(paddle1, paddle2, puck)

# メインゲームループ
clock = pygame.time.Clock()

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("無効なフレームを取得しました。")
        break

    # 画像の反転とRGB変換
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 手のランドマークの検出
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, landmark in enumerate(hand_landmarks.landmark):
                height, width, _ = image.shape
                cx, cy = int(landmark.x * width), int(landmark.y * height)
                hx = np.interp(cx, [0, width], [WIDTH, 0]) # x座標を正規化
                hy = np.interp(cy, [0, height], [0, HEIGHT]) # y座標を正規化

                if id == 9:  # 手のひらの中心点などを使ってパドルを制御
                    paddle1.move(hy)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        paddle2.move(paddle2.rect.y - 5)
    if keys[pygame.K_DOWN]:
        paddle2.move(paddle2.rect.y + 5)

    puck.update()

    # 衝突処理
    if pygame.sprite.collide_rect(puck, paddle1) or pygame.sprite.collide_rect(puck, paddle2):
        puck.speed_x = -puck.speed_x

    screen.fill(WHITE)
    all_sprites.draw(screen)
    pygame.display.flip()
    clock.tick(FPS)

    # 'q'キーが押された場合はループを終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# リソースの解放
cap.release()
cv2.destroyAllWindows()
pygame.quit()
sys.exit()
